In [4]:
import nltk as nl
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import WordNetLemmatizer
import pickle
import pandas as pd
from langdetect import detect
from collections import defaultdict

In [2]:
df=pd.read_csv('Dataset_final_filtered.tsv', sep='\t')

In [3]:
df

,document_ID,bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot_Values,Plot,NumberofPages,Publishing_Date,Characters,Setting,Url
0,0,MARS,NaN,Jasmine Rose,4.38,69,13,"{5: '47', 4: '7', 3: '11', 2: '2', 1: '2'}",❝�� my heart has become a planetand you are th...,NaN,2014,NaN,NaN,https://www.goodreads.com/book/show/23279048-mars
1,1,Black Box,NaN,Cassia Leo,4.02,6244,903,"{5: '2297', 4: '2320', 3: '1181', 2: '345', 1:...",♥️ Three fateful encounters....♥️ Two heart-br...,400.0,February 28th 2014,"['Mikki Gladstone', 'William ""Crush"" Slayer']","['Boston, Massachusetts']",https://www.goodreads.com/book/show/29539518-b...
2,2,Ruin and Rising,The Shadow and Bone Trilogy #3,Leigh Bardugo,4.09,158624,19396,"{5: '62107', 4: '59607', 3: '27962', 2: '6810'...",▶ \nAlternative Cover Edition #1\nThe capital ...,422.0,June 17th 2014,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/14061957-r...
3,3,Shadow and Bone,The Shadow and Bone Trilogy #1,Leigh Bardugo,3.99,313190,32760,"{5: '108466', 4: '121286', 3: '62748', 2: '146...",▶ \nAlternative Cover Edition #1\nSurrounded b...,358.0,June 5th 2012,"['Alina Starkov', 'Malyen Oretsev', 'Darkling'...",['Ravka '],https://www.goodreads.com/book/show/10194157-s...
4,4,An Ember in the Ashes,An Ember in the Ashes #1,Sabaa Tahir,4.27,184959,23147,"{5: '91602', 4: '63735', 3: '21164', 2: '5487'...",▶ \nAlternative Cover Edition #1\nLaia is a sl...,446.0,February 9th 2016,"['Laia', 'Elias Veturius', 'Helene Aquilla', '...",NaN,https://www.goodreads.com/book/show/27774758-a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26237,26237,Breathing Room,NaN,Marsha Hayles,4.19,1797,326,"{5: '794', 4: '632', 3: '306', 2: '51', 1: '14'}",\n\n\nEvvy Hoffmeister is thirteen years old w...,256.0,June 5th 2012,NaN,['Minnesota'],https://www.goodreads.com/book/show/13037526-b...
26238,26238,Eve of Destruction,Marked #2,S.J. Day,3.97,2764,135,"{5: '914', 4: '1066', 3: '607', 2: '132', 1: '...","\n\n\nClass is in, but Evangeline Hollis is st...",414.0,June 2nd 2009,NaN,['United States of America'],https://www.goodreads.com/book/show/2332154.Ev...
26239,26239,Time and Chance,Henry II and Eleanor of Aquitaine #2,Sharon Kay Penman,4.29,7829,356,"{5: '3699', 4: '2899', 3: '1085', 2: '115', 1:...","\n\n\n\nIn When Christ and His Saints Slept, a...",544.0,February 4th 2003,"['Thomas Becket', 'Eleanor of Aquitaine', 'Hen...",['England'],https://www.goodreads.com/book/show/234560.Tim...
26240,26240,An Autumn War,Long Price Quartet #3,Daniel Abraham,4.03,6331,248,"{5: '2143', 4: '2610', 3: '1264', 2: '224', 1:...",\n\n\n\nDaniel Abraham delighted fantasy reade...,366.0,July 22nd 2008,NaN,NaN,https://www.goodreads.com/book/show/2443516.An...


#  Dataset with only english plots above

In [ ]:

plots=df['Plot']  
for i in range(len(plots)):
    try:
        if detect(plots[i])=='en':
            continue
        else:
            df.drop(i,inplace=True)
    except:
        df.drop(i,inplace=True)

# Function used for clean the plot

In [5]:
lemma = WordNetLemmatizer()
def clean_info (string):
    # this command will split the string given in input in substrings by using 
    # the words given to RegexpTokenizer as argument
    

    
    # filter the punctuation
    tmp = nl.RegexpTokenizer(r"['\w-]+").tokenize(string)  
    
    #first we lower all the words otherwise words such as AND,IS,MY are not consider stopwords 
    tmp = [word.lower() for word in tmp]
    
     # filter the stopwords (e.g. 'the', 'my', 'an', ...)
    tmp = [word for word in tmp if not word in stopwords.words("english")]
    
    #we lemmatize all the words (e.g. 'dirn')
    lemma = WordNetLemmatizer()
    tmp = [lemma.lemmatize(word, pos = "v") for word in tmp]    # v for verbs
    tmp = [lemma.lemmatize(word, pos = "n") for word in tmp]    # n for nouns
    
    
    final = ' '.join(tmp)
    
    return final
    

# *CLEANED PLOTS* used after for the vocabulary


In [ ]:
cleaned_plots=[]
n=len(plots)

for i in range(n):
    p=clean_info(plots[i])
    cleaned_plots.append(p)

with open('clean_plots.pkl', 'wb') as f:
    pickle.dump(cleaned_plots, f, pickle.HIGHEST_PROTOCOL)
    f.close()
    

# *VOCABULARY*

In [ ]:
#we map all terms with number 
#the map function is simple , each word is associated to a number from 1 to n(number of words we have)

vocabulary={}
index=1

for p in cleaned_plots:
    plot=p.split(' ')
    for word in plot:
        if word not in vocabulary:
            vocabulary[word]=index
            index+=1

with open('vocabulary.pkl', 'wb') as f:
    pickle.dump(vocabulary, f, pickle.HIGHEST_PROTOCOL)
    f.close()

# INVERTED INDEX

In [ ]:
documents_ID=df['document_ID']
# create an inverted index without tfidf
'''
ID={word_id:[doc_1,doc_2,.....]}
'''
n=len(cleaned_plots)
ID=defaultdict(list)

for i in range(n):
    p=cleaned_plots[i].split(' ')
    p=set(p)
    for word in p:
        ID[vocabulary[word]].append(documents_ID[i])    

with open('ID.pkl', 'wb') as f:
    pickle.dump(ID, f, pickle.HIGHEST_PROTOCOL)
    f.close()
       

# INVERTED INDEX with tfidf

In [ ]:
#create an ID with tfidf
'''
ID_tfifd={word_id:[(doc1_ID,tfidf1),(doc2_ID,tfidf2......)]}
'''


def calculate_tfidf(doc_id,w,N):
    #find the plot
    plot_base=cleaned_plots[doc_id]
    
    #count the time of a word appears in the plot
    time_frequency=plot_base.count(w)
    
    #calculate the tfidf
    inv_doc_freq=np.log10(n/N)
    
    return round(time_frequency*inv_doc_freq,4)

#global value for tfidf
#total number of documents
n=len(df['document_ID']) 

ID_tfidf=defaultdict(list)

for word in vocabulary:
    word_mapped=vocabulary[word]
    n_ID=len(ID[word_mapped])   #local value necessary for tfidf
    for document_ID in ID[word_mapped]:
        ID_tfidf[word_mapped].append((document_ID,calculate_tfidf(document_ID,word,n_ID)))
        
with open('ID_with_tfidf.pkl', 'wb') as f:
    pickle.dump(ID_tfidf, f, pickle.HIGHEST_PROTOCOL)
    f.close()

# Function useful for similarity

In [ ]:
#mapping a plot in number using vocabulary and transform it in an array for fast counting operations
'''
return an array (0,0,2,0,3,0,.........0,1,0)
for each plot which represents the number of times each word 
of the vocabulary appears .
The first value is always 0 because the vocabulary starts from 1 so we need to 
increase the dimension of 1,otherwise if the last word of the vocabulary 
is in the plot ,when we update its value we give an error.
'''
l_v=len(vocabulary)+1 #because vocabulary starts from 1
def map_plot (plot):
    d_pp=plot.split(' ')
    temp=np.zeros(l_v)
    for w in d_pp:
        temp[vocabulary[w]]+=1
    return temp



# Function for search query

In [ ]:
# we will need first these two functions to implement the search function 


# this function takes as input a list of lists and gives back the index of the list that has minimus first element

def find_min_list (L):
    min_elem = L[0][0]
    count = 0
    index = 0
    for l in L[1:]: 
        count+=1
        if min_elem > l[0]:
            index = count 
            min_elem = l[0] 

    return index 
        

# this function takes as input a list of lists and gives back the list created from the intersection of the lists 
    
def intersect_list (L):
    results = []
    while all(len(l) > 0 for l in L):

        if all([L[0][0] == l[0] for l in L[1:]]):     
            results.append(L[0][0])
            L = [l[1:] for l in L]

        else : 
            min_index = find_min_list(L)
            L[min_index] = L[min_index][1:]

    return results